In [ ]:
# export
from fastai.datasets import URLs, untar_data
from pathlib import Path
import torch, re, PIL, os, mimetypes, csv
import matplotlib.pyplot as plt
from collections import OrderedDict
from typing import *
import pandas as pd, numpy as np

## Data block API from config class

### Core helper functions

In [ ]:
# export
def listify(o):
    if o is None: return []
    if isinstance(o, list): return o
    if isinstance(o, str): return [o]
    if isinstance(o, Iterable): return list(o)
    return [o]

In [ ]:
# export
def compose(x, funcs, *args, order_key='_order', **kwargs):
    key = lambda o: getattr(o, order_key, 0)
    for f in sorted(listify(funcs), key=key): x = f(x, **kwargs)
    return x

In [ ]:
# export
def uniqueify(x, sort=False):
    res = list(OrderedDict.fromkeys(x).keys())
    if sort: res.sort()
    return res

In [ ]:
# export
def setify(o): return o if isinstance(o,set) else set(listify(o))

def _get_files(p, fs, extensions=None):
    p = Path(p)
    res = [p/f for f in fs if not f.startswith('.')
           and ((not extensions) or f'.{f.split(".")[-1].lower()}' in extensions)]
    return res

def get_files(path, extensions=None, recurse=False, include=None):
    path = Path(path)
    extensions = setify(extensions)
    extensions = {e.lower() for e in extensions}
    if recurse:
        res = []
        for i,(p,d,f) in enumerate(os.walk(path)): # returns (dirpath, dirnames, filenames)
            if include is not None and i==0: d[:] = [o for o in d if o in include]
            else:                            d[:] = [o for o in d if not o.startswith('.')]
            res += _get_files(p, f, extensions)
        return res
    else:
        f = [o.name for o in os.scandir(path) if o.is_file()]
        return _get_files(path, f, extensions)

### Processor and ItemGetter

In [ ]:
# export
class Processor():
    def __call__(self, items):  return items
    def process1(self, item):   return item
    def deprocess1(self, item): return item
    def deprocess(self, items): return items

In [ ]:
# export
class ItemGetter():
    _default_proc = None
    def __init__(self, procs=None): 
        self.procs = [p() for p in listify(self._default_proc)] if procs is None else procs
    def __call__(self, items): return compose(items, self.procs)
    
    def get(self, o): return o  #How to get the actual item from o (example: fn -> Image, idxs -> 1hot encoded tensor)
    def raw(self, o): return o  #Undoes get when needed for representation (1hot encoded tensor -> idxs)
    
    def show(self, x, ax):                       raise NotImplementedError #How to show one element
    def show_xys(self, xs, ys, y_get, **kwargs): raise NotImplementedError #How to show a bunch of xs and ys

### Data block API core

One ItemList to contain them all

In [ ]:
# export
class ItemList():
    def __init__(self, items, item_get=None, tfms=None):
        if item_get is None: item_get = ItemGetter()
        self.item_get,self.tfms = item_get,tfms
        self.items = self.item_get(listify(items))
    def _get(self, i): return compose(self.item_get.get(i), self.tfms)
    def __getitem__(self, idx):
        try: return self._get(self.items[idx])
        except TypeError:
            if isinstance(idx[0],bool):
                assert len(idx)==len(self) # bool mask
                return [self._get(o) for m,o in zip(idx,self.items) if m]
            return [self._get(self.items[i]) for i in idx]
    def __len__(self): return len(self.items)
    def __iter__(self): return iter(self.items)
    def __setitem__(self, i, o): self.items[i] = o
    def __delitem__(self, i): del(self.items[i])
    def __repr__(self):
        res = f'{self.__class__.__name__} ({len(self)} items)\n{self.items[:10]}'
        if len(self)>10: res = res[:-1]+ '...]'
        return res
    
    def new(self, items, cls=None): #Unused for now... TODO remove?
        if cls is None: cls=self.__class__
        return cls(items, tfms=self.tfms)
    
    def deproc(self, item):
        item = self.item_get.raw(item)
        for proc in reversed(listify(self.item_get.procs)):
            item = proc.deproc1(item)
        return item
    
    def obj(self, idx):
        isint = isinstance(idx, int) or (isinstance(idx,torch.LongTensor) and not idx.ndim)
        item = self[idx]
        return self.deproc(item) if isint else [self.deprocess(o) for o in item]

LabeledData is just a convenience wrapper other xs and ys

In [ ]:
# export
class LabeledData():
    def __init__(self, x, y, tfms=None):  self.x,self.y,self.tfms = x,y,tfms
    def __repr__(self):        return f'{self.__class__.__name__}\nx: {self.x}\ny: {self.y}\n'
    def __getitem__(self,idx): return compose((self.x[idx],self.y[idx]), self.tfms)
    def __len__(self):         return len(self.x)

Dataloader and DataBunch

In [ ]:
# export
from torch.utils.data.dataloader import DataLoader
def get_dl(ds, bs, shuffle=False, drop_last=False, **kwargs):
    return DataLoader(ds, batch_size=bs, shuffle=shuffle, drop_last=drop_last, **kwargs)

In [ ]:
# export
class DataBunch():
    def __init__(self, train_dl, valid_dl):
        self.train_dl,self.valid_dl = train_dl,valid_dl
        
    @property
    def train_ds(self): return self.train_dl.dataset
    @property
    def valid_ds(self): return self.valid_dl.dataset
    
    def show_batch(self, is_valid=False, items=9, **kwargs):
        x,y = next(iter(self.valid_dl if is_valid else self.train_dl))
        self.train_ds.x.item_get.show_xys(
            [self.train_ds.x.deproc(x[i]) for i in range(items)], 
            [self.train_ds.y.deproc(y[i]) for i in range(items)], 
            self.train_ds.y.item_get,
            **kwargs)

Main class to represent any kind of data. User provides the `get_x_cls` and `get_y_cls` then the four functions. At init everything is constructed with optionals transforms or custom instances of `get_x`/`get_y`.

In [ ]:
# export
class DataBlock():
    get_x_cls = ItemGetter
    get_y_cls = ItemGetter
    def get_source(self):         raise NotImplementedError
    #Return the source of your data (path, dataframe...) and optionally download it
    def get_items(self, source):  raise NotImplementedError
    #Use source to return the list of all items
    def split(self, items):       raise NotImplementedError
    #Explain how so split the items. Return two list of integers or two boolean masks
    def label(self, items):       raise NotImplementedError
    #Explain how to label your items. Return a list of labels
        
    def __init__(self, tfms_x=None, tfms_y=None, tfms_xy=None, get_x=None, get_y=None):
        self.source = self.get_source()
        items = ItemList(self.get_items(self.source)) #Just for fancy indexing
        split_idx = self.split(items)
        labels = ItemList(self.label(items))
        if get_x is None: get_x = self.get_x_cls()
        if get_y is None: get_y = self.get_y_cls()
        x_train,x_valid = map(lambda o: ItemList(items[o],  item_get=get_x, tfms=tfms_x), split_idx)
        y_train,y_valid = map(lambda o: ItemList(labels[o], item_get=get_y, tfms=tfms_y), split_idx)
        self.train = LabeledData(x_train, y_train, tfms_xy)
        self.valid = LabeledData(x_valid, y_valid, tfms_xy)
    
    def databunch(self, bs=64, **kwargs):
        dls = [get_dl(ds, bs, shuffle=s, drop_last=s, **kwargs) for (ds, s) in zip([self.train, self.valid], [True,False])]
        return DataBunch(*dls)

### First ItemGetters

How to get images

In [ ]:
# export
class ImageGetter(ItemGetter):
    def __init__(self, procs=None, cmap=None, alpha=1.): 
        super().__init__(procs)
        self.cmap,self.alpha = cmap,alpha
    def get(self, fn): return PIL.Image.open(fn)
    def show(self, x, ax):
        ax.imshow(x[0] if x.shape[0] == 1 else x.permute(1,2,0), cmap=self.cmap, alpha=self.alpha)
        ax.axis('off')
    def show_xys(self, xs, ys, y_get, cols=3, figsize=None):
        rows = (len(xs) // cols if len(xs)%cols == 0 else len(xs)//cols+1) 
        if figsize is None: figsize = (cols*3, rows*3)
        fig,axs = plt.subplots(rows, cols, figsize=figsize)
        for x,y,ax in zip(xs, ys, axs.flatten()):
            self.show(x, ax)
            y_get.show(y, ax)

How to get categories

In [ ]:
# export
class CategoryProcessor(Processor):
    def __init__(self): self.vocab=None
    
    def __call__(self, items):
        #The vocab is defined on the first use.
        if self.vocab is None:
            self.vocab = uniqueify(items)
            self.otoi  = {v:k for k,v in enumerate(self.vocab)}
        return [self.proc1(o) for o in items]
    def proc1(self, item):  return self.otoi[item]
    
    def deprocess(self, idxs):
        assert self.vocab is not None
        return [self.deproc1(idx) for idx in idxs]
    def deproc1(self, idx): return self.vocab[idx]

In [ ]:
# export
class CategoryGetter(ItemGetter):
    _default_proc = CategoryProcessor
    def show(self, x, ax): ax.set_title(x)

### Helper functions

To get image files

In [ ]:
# export
image_extensions = set(k for k,v in mimetypes.types_map.items() if v.startswith('image/'))

In [ ]:
# export
def get_image_files(path, include=None):
    return get_files(path, extensions=image_extensions, recurse=True, include=include)

To split

In [ ]:
# export
def random_splitter(items, valid_pct=0.2, seed=None): 
    if seed is not None: torch.manual_seed(seed)
    rand_idx = torch.randperm(len(items))
    cut = int(valid_pct * len(items))
    return rand_idx[:cut],rand_idx[cut:] 

In [ ]:
# export
def _grandparent_mask(items, name):
    return [(o.parent.parent.name if isinstance(o, Path) else o.split(os.path.sep)[-2]) == name for o in items]

def grandparent_splitter(items, train_name='train', valid_name='valid'):
    return _grandparent_mask(items, train_name),_grandparent_mask(items, valid_name)

To label

In [ ]:
# export
def parent_labeller(items):
    return [o.parent.name if isinstance(o, Path) else o.split(os.path.sep)[-1] for o in items]

In [ ]:
# export
def func_labeller(items, func):
    return [func(o) for o in items]

In [ ]:
# export
def re_labeller(items, pat):
    pat = re.compile(pat)
    def _inner(o):
        res = pat.search(str(o))
        assert res,f'Failed to find "{pat}" in "{s}"'
        return res.group(1)
    return func_labeller(items, _inner)

### First data:

In [ ]:
class PetsData(DataBlock):
    get_x_cls = ImageGetter
    get_y_cls = CategoryGetter
    
    def get_source(self):        return untar_data(URLs.PETS)
    def get_items(self, source): return get_image_files(source/"images")
    def split(self, items):      return random_splitter(items)
    def label(self, items):      return re_labeller(items, pat = r'/([^/]+)_\d+.jpg$')

In [ ]:
data = PetsData()

In [ ]:
img,cls = data.train[0]

In [ ]:
img

In [ ]:
cls

In [ ]:
data.train.y.obj(0)

In [ ]:
# export
def make_rgb(item): return item.convert('RGB')

In [ ]:
# export
class Transform(): _order=0

class ResizeFixed(Transform):
    _order=10
    def __init__(self,size,mode=PIL.Image.BILINEAR):
        if isinstance(size,int): size=(size,size)
        self.size,self.mode = size,mode
        
    def __call__(self, item): return item.resize(self.size, self.mode)

def to_byte_tensor(item):
    res = torch.ByteTensor(torch.ByteStorage.from_buffer(item.tobytes()))
    w,h = item.size
    return res.view(h,w,-1).permute(2,0,1)
to_byte_tensor._order=20

def to_float_tensor(item): return item.float().div_(255.)
to_float_tensor._order=30

In [ ]:
tfms = [make_rgb, ResizeFixed(128), to_byte_tensor, to_float_tensor]

In [ ]:
data = PetsData(tfms_x=tfms)

In [ ]:
datab = data.databunch()

In [ ]:
datab.show_batch()

In [ ]:
x,y = next(iter(datab.train_dl))

In [ ]:
x.shape

## Try different data

### MNIST

In [ ]:
class MnistData(DataBlock):
    get_x_cls = ImageGetter
    get_y_cls = CategoryGetter
    
    def get_source(self):        return untar_data(URLs.MNIST)
    def get_items(self, source): return get_image_files(source)
    def split(self, items):      return grandparent_splitter(items, train_name='training', valid_name='testing')
    def label(self, items):      return parent_labeller(items)

In [ ]:
data = MnistData(tfms_x=[to_byte_tensor, to_float_tensor]).databunch()

In [ ]:
data.show_batch()

In [ ]:
data.train_ds.x.item_get.cmap='gray'

In [ ]:
data.show_batch()

### Planet

In [ ]:
path = untar_data(URLs.PLANET_SAMPLE)

In [ ]:
path.ls()

In [ ]:
df = pd.read_csv(path/'labels.csv')

In [ ]:
df.head()

In [ ]:
# export
def onehot(x, c):
    res = torch.zeros(c)
    res[x] = 1.
    return res

In [ ]:
# export
class MultiCategoryProcessor(CategoryProcessor):
    def __call__(self, items):
        #The vocab is defined on the first use.
        if self.vocab is None:
            vocab = set()
            for c in items: vocab = vocab.union(set(c))
            self.vocab = list(vocab)
            self.vocab.sort()
            self.otoi  = {v:k for k,v in enumerate(self.vocab)}
        return [self.proc1(o) for o in items]
    def proc1(self, item):  return [self.otoi[o] for o in item if o in self.otoi]
    
    def deproc1(self, idx): return [self.vocab[i] for i in idx]

class MultiCategoryGetter(ItemGetter):
    _default_proc = MultiCategoryProcessor
    
    def get(self, o): return onehot(o, len(self.procs[0].vocab))
    def raw(self, o): return [i for i,x in enumerate(o) if x == 1]
    def show(self, x, ax): ax.set_title(';'.join(x))

In [ ]:
# export
def read_column(df, col_name, prefix='', suffix='', delim=None):
    values = df[col_name].values.astype(str)
    values = np.char.add(np.char.add(prefix, values), suffix)
    if delim is not None:
        values = np.array(list(csv.reader(values, delimiter=delim)))
    return values

In [ ]:
class PlanetData(DataBlock):
    get_x_cls = ImageGetter
    get_y_cls = MultiCategoryGetter
    
    def get_source(self):        
        self.path = untar_data(URLs.PLANET_SAMPLE)
        return pd.read_csv(path/'labels.csv')
    def get_items(self, source): return read_column(source, 'image_name', prefix=f'{self.path}/train/', suffix='.jpg')
    def split(self, items):      return random_splitter(items)
    def label(self, items):      return read_column(self.source, 'tags', delim=' ')

In [ ]:
data = PlanetData(tfms_x=tfms).databunch()

In [ ]:
data.show_batch()

### Camvid

In [ ]:
# export
class SegmentMaskGetter(ImageGetter):
    def __init__(self, procs=None, cmap='tab20', alpha=0.5): 
        super().__init__(procs, cmap=cmap, alpha=alpha)

In [ ]:
class CamvidData(DataBlock):
    get_x_cls = ImageGetter
    get_y_cls = SegmentMaskGetter
    
    def get_source(self):        return untar_data(URLs.CAMVID_TINY)      
    def get_items(self, source): return get_image_files(source/'images')
    def split(self, items):      return random_splitter(items)
    def label(self, items):      
        path_lbl = self.source/'labels'
        codes = np.loadtxt(self.source/'codes.txt', dtype=str)
        return func_labeller(items, lambda x: path_lbl/f'{x.stem}_P{x.suffix}')

In [ ]:
# export
def make_mask(item): return item.convert('L')
def to_long_tensor(item): return item.long()
to_long_tensor._order=30

In [ ]:
tfms_x = [make_rgb,  ResizeFixed(128), to_byte_tensor, to_float_tensor]
tfms_y = [make_mask, ResizeFixed(128, mode=PIL.Image.NEAREST), to_byte_tensor, to_long_tensor]

In [ ]:
data = CamvidData(tfms_x=tfms_x, tfms_y=tfms_y).databunch(bs=16)

In [ ]:
data.show_batch()

## Export

In [ ]:
! python notebook2script.py "200_datablock_config.ipynb"